# BME 590: Lab 2 - Food Coloring Academy
## In this tutorial, we will build on the skills learned in lab 1 and perform some operations with a multi-channel pipette.
#### Thank you to Maggie Gatongi for helping with the design of this notebook and being the first graduate of food coloring academy. You rock! :)


Recall, we begin by importing `LiquidHandler`, a backend called `ChatterBoxBackend` that prints the text output of our commands, a class `Visualizer` that provides a visualization of the robot deck as we run commands, and a class `OTDeck` that will represent the deck of an OpenTrons OT2.

Make sure to also `import opentrons` !

### Imports

In [13]:
import pylabrobot

from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends import ChatterBoxBackend
from pylabrobot.visualizer.visualizer import Visualizer
from pylabrobot.resources.opentrons import OTDeck

from pylabrobot.resources.opentrons.load import *
from pylabrobot.resources.opentrons.plates import *

from pylabrobot.resources import set_tip_tracking, set_volume_tracking, set_cross_contamination_tracking
set_tip_tracking(True), set_volume_tracking(True)

# Optional, use when interested in protecting against accidental cross contamination
set_cross_contamination_tracking(True)

import opentrons
import time

### Set up the Deck and Visualizer

In [ ]:
# Make sure to use the ChatterBoxBackend() and the OTDeck()
lh = LiquidHandler(backend=ChatterBoxBackend(), deck=OTDeck())

await lh.setup()

vis = Visualizer(resource=lh)
await vis.setup()

### Add labware to the deck

In [4]:
from pylabrobot.resources import (
    corning_96_wellplate_360ul_flat,
    opentrons_24_tuberack_eppendorf_2ml_safelock_snapcap,
    opentrons_96_tiprack_300ul,
    nest_12_reservoir_15ml,
    nest_96_wellplate_2ml_deep
)

We'll begin this lesson by using our skills from last time and setting up a deck. We are going to model the transfer of liquids which start in 2mL eppendorf tubes to a 96-well plate. Here's a refresher on how to build a deck!

In [ ]:
# Let's add a tube rack to spot 4
# Declare a tube rack object from the labware we imported
tube_rack = opentrons_24_tuberack_eppendorf_2ml_safelock_snapcap("tube_rack")

# Assign the tube rack to spot 4 of the deck
lh.deck.assign_child_at_slot(tube_rack, 4)

# Declare a tip rack object
tip_rack =  opentrons_96_tiprack_300ul("tip_rack_1")

# Assign the tip rack to spot 7
lh.deck.assign_child_at_slot(tip_rack, 7)

# Declare a plate object
plate =  corning_96_wellplate_360ul_flat("prep_plate")

# Assign the plate to spot 1
lh.deck.assign_child_at_slot(plate, 1)

### 1.0.5 Adding Liquids to the Deck

Let's manually load some liquids to our `tube_rack`. In the real world, this is something you would do as preparation for running an automatic protocol. We can add up to the `max_volume` of the tube. If you go over this number, **PyLabRobot** will throw an error. We shall add 2000µL of 4 different dyes to the first column of our `tube_rack`. This corresponds to wells *A1, B1, C1, and D1*.

To iterate over locations on labware, we use the `traverse()` function. This produces a [generator object](https://www.geeksforgeeks.org/generators-in-python/) that we use the `next` keyword on to yield our desired wells.

`traverse()` takes in two arguments: **batch_size** is the amount of wells to return, and **direction** is how to iterate over the wells. In our case, we use `"down"` to return the wells column wise.

<details>
    <summary>More info on <b>direction:</summary>

* `"down"`, `"snake_down"`, `"right"`, and `"snake_right"` start at the top left item **(A1)**.
        
* `"up"` and `"snake_up"` start at the bottom left **(H1)**.
    
* `"left"` and `"snake_left"` start at the top right **(A12)**.

* The `snake` directions alternate between going in the given direction and going in the opposite direction. For example, `"snake_down"` will go from A1 to H1, then H2 to A2, then A3 to H3, etc.
</details>

In [ ]:
first_col_tubes = next(tube_rack.traverse(batch_size=4, direction='down'))
first_col_tubes

Use the `tracker.set_liquids()` function to put liquid in a `tube`. The `tracker` class contains all of the methods associated with keeping record of how much/what kind of liquid is in a given container.

Pass in a string for **"Liquid_Type"** and a number for **Volume** to this function.

Let's add our dyes to the tubes in `first_col_tubes`.

In [7]:
for i in range(len(first_col_tubes)):
    # [(liquid, volume)]
    first_col_tubes[i].tracker.set_liquids([(f"Dye_{i}", 2000)])

##### Here's a Utility function for printing all of the filled spots of any labware.

In [8]:
def print_filled_spots_of_labware(labware):
    
    # Retrieve all children within the specified labware (tubes of a tube rack, wells of a wellplate, etc)
    # Note, for this explanation I am using the word "tube" to refer to the children of the parent labware
    # Actually, this can be anything, so if you call the function on a well plate it will return the individual wells!

    # Get all children of the input labware
    all_tubes = labware.get_all_children()

    # Flag to check if all tubes are empty
    all_empty = True

    # Iterate through each tube in the labware
    for tube in all_tubes:
        # Access the current status of liquids in the tube
        liquid = tube.tracker.liquids

        # Check if the tube contains any liquid
        if liquid != []:
            # Print the position and contents of the tube
            print(f"Spot {tube.name.split('_')[-1]} contains:")

            # Extract and store the name of the liquid
            name = liquid[0][0]
            # Extract and store the volume of the liquid
            vol = liquid[0][1]

            # Output the volume and name of the liquid in the tube
            print(f"{vol}uL of {name}")

            # Update the flag since this tube is not empty
            all_empty = False
            
    # Check if all tubes were empty and print message if true
    if all_empty:
        print("Entire labware is empty!")

##### Let's give it a call!

In [ ]:
print_filled_spots_of_labware(tube_rack)

In [ ]:
# Try calling the function on your plate!
# Hint, pass the plate in as a parameter
# Hint, Hint, make sure the name of the function is correct ;)
print_fi1led_spot_of_1abware(???)

### 1.0.6 Moving Liquids from Point A to Point B

Now that we've added some dyes to our tube rack, let's use the robot to move some liquid to our `plate`. The first step of a liquid transfer is acquiring a tip.

You can acquire a tip by calling `lh.pick_up_tips()`, and passing in the `TipSpots` of the tips you want to retrieve. `TipSpots` are indexed the same way that wells are.

When a tip is picked up from a `TipRack`, it's location will turn white on the visualizer. To see what tips are currently on the robot, call `lh.head`. This returns a dictionary where the keys are the indices of the different channels of the main pipettor, and where the values are instances of the `TipTracker` class, allowing you to get information about how tips move on and off of a given channel.

If you want to reset the state of tips on a robot, call `lh.return_tips()`. This function will automatically return the tips to their original locations.

In [ ]:
# Careful! Don't call this function unless you currently have tips on the robot. If you attempt to call
# return_tips() on an empty head, an error will occur!
await lh.return_tips()

Let's try picking up 8 tips along the diagonal.

In [ ]:
await lh.pick_up_tips(tip_rack["A1", "B2", "C3", "D4", "E5", "F6", "G7", "H8"])

Rather than use `lh.return_tips()` all of the time, you can also call `lh.drop_tips()` and pass in specifically where to place the tip, and what channel's tip to drop.

In [ ]:
await lh.drop_tips(tip_spots = tip_rack["A1", "B2", "C3", "D4", "E5", "F6", "G7", "H8"],
                   use_channels = [0,1,2,3,4,5,6,7])

The order in which you pass in the `tip_spots` and the `use_channels` lists will determine which channel gets which tip. Take a look!

In [ ]:
await lh.pick_up_tips(tip_spots = tip_rack["A1", 'C3', "E7"], use_channels = [2,0,1])

In [ ]:
lh.head

##### Here are some utility functions that will print the status of tips on the pipetter. If there is a tip on a channel, this function will output its origin.

In [2]:
def print_channels_tip_origin(lh):
    # Prints the origin location of all tips currently on the robot
    cur_pipetter = lh.head

    for channel in cur_pipetter:
        print(f"Channel {channel}:")

        tip_tracker = lh.head[channel]
        
        if tip_tracker.has_tip == True:
            print(tip_tracker.get_tip_origin())
        else:
            print("No tip present.")
        print()

In [3]:
def print_channel_status(lh):
    # Prints the status of liquids, if present, in each channel
    cur_pipetter = lh.head

    for channel in cur_pipetter:
        print(f"Channel {channel}:")

        tip_tracker = lh.head[channel]
        
        if tip_tracker.has_tip == True:
            tip = tip_tracker.get_tip()
            print(tip.tracker.liquids)
        else:
            print("No tip present.")
        print()

##### Try it out!

In [ ]:
print_channels_tip_origin(lh)

In [ ]:
print_channel_status(lh)

##### You also don't need to specify channels in order. If you wanted to skip channel 3, we can do so by just passing in the next channel index and skipping 3.

In [ ]:
await lh.pick_up_tips(tip_rack["D3"], use_channels = [4])
print("\n")
print(lh.head[4])
print("\n")
time.sleep(2)
await lh.return_tips()

In [ ]:
await lh.drop_tips(tip_spots = tip_rack["E7"], use_channels = [2])

##### If you want to check if a spot in the tip rack has a tip, call `TipSpot.has_tip()`. This could be useful if you are writing a script with many operations and want to automatically calculate where you should grab your next tip from.

In [ ]:
tip_rack["A1"][0].has_tip()

#### Let's do a real liquid transfer!

In [ ]:
# Let's try doing a real liquid transfer now! We'll move 200 uL of dye_0 into the plate.

# Grab a tip from A1 of the tip rack
await lh.pick_up_tips(tip_rack["A1"])

# Aspirate 200 uL of dye from A1 of the tube rack
await lh.aspirate(tube_rack["A1"], vols=[200])
time.sleep(2)

# Dispense 200 uL of Dye_1 in A1 of the plate
await lh.dispense(plate["A1"], vols=[200])
time.sleep(2)

# Return tips to the rack. Note, in a real experiment you would discard the tips.
# await lh.discard_tips()
await lh.return_tips()

In [ ]:
print_filled_spots_of_labware(plate)

#### Cross Contamination Demo

In [ ]:
# PyLabRobot has a built in feature that will save you if you accidentally aspirate
# from a well with a contaminated tip. Recall that the tip in A1 was used for dye_0.
# Let's see what happens when we try aspirating dye_1 with the same tip!

await lh.pick_up_tips(tip_rack["A1"])
time.sleep(1)

await lh.aspirate(tube_rack["B1"], vols=[200])
time.sleep(1)

await lh.dispense(plate["B1"], vols=[200])
time.sleep(1)

await lh.return_tips()

## 1.1 Basic Liquid Handling Exercises:

Complete the following operations using the deck we created in the tutorial. As a reminder, there should be a `tip rack on spot 7`, a `tube rack on spot 4`, and a `96 well plate on spot 1`.

1. Add `100 uL of Dye_1` to well `A1`.
2. Add `200 uL of Dye_2` to well `B7`.
3. Add `50 uL of Dye_1` **and** `50 uL of Dye_3` to well `C9`.
4. Add `25 uL of Dye 1`, `25 uL of Dye 2`, `25 uL of Dye 3`, and `25 uL of Dye 4` to well `D12`.
5. Add `50 uL of Dye 1` and `50 uL of the solution in D12` to well `A3`.
6. Add `1800 uL of Dye 1` to well `F12`. *Hint, you have dye on the plate already*

**Use multiple tips as needed to avoid `Cross Contamination`**.

**Remember to discard used tips in the trash can when finished!**

In [5]:
# Problem 1

In [6]:
# Problem 2

In [7]:
# Problem 3

In [8]:
# Problem 4

In [9]:
# Problem 5

In [10]:
# Problem 6

## 1.2.0 Advanced Liquid Handling: Mixing liquids and making new compounds!

Now that we've established how to move liquids around, let's try something a little more interesting. In this section, you will experiment with mixing various colors of dye to make a rainbow!

### Reset the deck

Recall that the command to remove labware is `lh.deck.unassign_child_resource(LABWARE)`

In [ ]:
# Make sure that you remove all of the labware currently on the deck
lh.deck.unassign_child_resource(tube_rack)
lh.deck.unassign_child_resource(plate)
lh.deck.unassign_child_resource(tip_rack)

### Add food coloring stocks to the deck

In [ ]:
# To practice moving liquids from different locations, 
# we will place each dye in its own labware. This is horribly inefficient
# in practice, but for this tutorial is a good way to learn how to index 
# different labware.

blue_reservoir = nest_96_wellplate_2ml_deep("blue_reservoir")
lh.deck.assign_child_at_slot(blue_reservoir, 7)

red_reservoir = nest_96_wellplate_2ml_deep("red_reservoir")
lh.deck.assign_child_at_slot(red_reservoir, 8)

yellow_reservoir = nest_12_reservoir_15ml("yellow_reservoir")
lh.deck.assign_child_at_slot(yellow_reservoir, 4)

prep_plate = corning_96_wellplate_360ul_flat("prep_plate")
lh.deck.assign_child_at_slot(prep_plate, 1)

tr_1 = opentrons_96_tiprack_300ul("tr_1")
lh.deck.assign_child_at_slot(tr_1, 6)

#### Key Python Concepts:
* [for loops](https://www.geeksforgeeks.org/python-for-loops/) - Useful when you need to iterate over many items with a defined numerical structure.
* [f-strings](https://www.geeksforgeeks.org/formatted-string-literals-f-strings-python/) - Useful when you need to change the contents of a string according to some variable, see below for example.

In [27]:
# Since we are simulating multi-channel pipette use, we
# will add dye to the entire first column

# this is an array of all of the capital letters
# save this code as it may help you later!

cptl_alphabet = [chr(i) for i in range(65, 91)]

for i in range(8):
    # Using the f-string allows us to iterate through 
    # wells A1 -> B1 -> C1 -> ... -> H1
    # all in one line of code!
    blue_reservoir[f'{cptl_alphabet[i]}1'][0].tracker.set_liquids([("Blue Dye", 2000)])
    red_reservoir[f'{cptl_alphabet[i]}1'][0].tracker.set_liquids([("Red Dye", 2000)])

In [28]:
# The yellow reservoir needs a slightly different treatment
# to load it. Its wells are all row A, and there is only
# 1 well per column to fill

yellow_reservoir['A4'][0].tracker.set_liquids([("Yellow Dye", 2000)])

### Exercise 1: Two red columns

For your first task, write and execute code that adds 200 uL of `Red Dye` to columns 1 and 2 of the `Plate`.

### Exercise 2: Two blue columns

Now, write and execute code that adds 200 uL of `Blue Dye` to columns 3 and 4 of the `Plate`.

### Exercise 3: Moving liquids from one end of the plate to the other

Pretty fun stuff, right? Now let's work on an intra-plate transfer. Using the dye already on the `Plate`, add 50 uL of `Red Dye` to column 5, and 50 uL of `Blue Dye` to column 6.

### Exercise 4: Purple Party! Mixing liquids

Alright, let's do something crazy. Using dye already on the `Plate`, make 100 uL of `Purple Dye` in column 6. To make `Purple Dye`, add equal parts of `Red Dye` and `Blue Dye` to a well. Make sure to `mix` the solution so that the dyes combine into the new color.

* Note, you will have to use your imagination here. `PyLabRobot` doesn't have official dye support yet, and all liquids appear red. Joe is going to work on this later!

#### Mixing Hint!

##### Quick reference on [functions](https://www.geeksforgeeks.org/python-functions/)

In [ ]:
# A mixing operation is just a repeated set of aspirate
# and dispense instructions. You can write this as your
# own Python function, or just write a for loop each time.
# For your own sake and sanity, we recommend writing a function!

num_mixes = 4

await lh.pick_up_tips(tr_1["A1"])
time.sleep(1)

for _ in range(num_mixes):
    await lh.aspirate(plate["B1"], vols=[200])
    time.sleep(1)
    await lh.dispense(plate["B1"], vols=[200])

await lh.discard_tips()

#### Back to the exercise!

### Exercise 5: Green and Orange

Using the same code you used in exercise 4, make 100uL of `Green Dye` in column 7, and 100uL of `Orange Dye` in column 8. Think about how you would abstract this process into functions. You will need to do this later, and it is a good habit to get in to!

### Exercise 6: Putting it all together

Now's the time to put your Python and PyLabRobot skills to the test! Using the functions you've been writing along the way, write a function that creates the entire plate we generated. Your function should take in at least the following parameters:
1. Red Dye Source
2. Blue Dye Source
3. Yellow Dye Source
4. Destination Plate

Your main `make_rainbow_plate` function will have subsequent calls to smaller functions. For instance, first you will call `add_red_dye` on the first two columns, then `add_blue_dye` on the second two columns etc. Challenge yourself to not have a massive paragraph of code. Breaking these smaller tasks into their own functions is a great skill that will be useful in later exercises!

### Challenge:

Remember building a deck for a food coloring serial dilution in lab 1? Using your new liquid handling skills, we are going to make a rainbow! Using the deck from before, you must prepare in a 96 well plate a serial dilution of the following colors: red, orange, yellow, green, blue, purple.
* Assume that the starting concentration of our dyes are **1M**
* Your concentrations should be: 500mM, 250mM, 125mM, 62.5mM, 31.25mM, and 15.63mM.
* The final volume of dye in each well should be **200 uL.**
* Recall the Dilution Equation of the form $C_{1}V_{1} = C_{2}V_{2}\$